In [6]:
# Import required libraries
import pandas as pd

try:
    # Read the CSV file from the specified path
    # Note: Using raw string (r) to handle Windows file paths
    df = pd.read_csv(r'data/Results.csv')
    print("CSV file successfully loaded!")
    
except FileNotFoundError:
    print("Error: The file was not found. Please check if the path 'E:\\Dropbox\\AI\\Data\\Radiology_Reports\\radiology_reports.csv' is correct.")
except pd.errors.EmptyDataError:
    print("Error: The CSV file is empty.")
except Exception as e:
    print(f"An unexpected error occurred while reading the file: {str(e)}")

print(df.head())


CSV file successfully loaded!
              MRN           EXAM DATE/TIME            VERIFIED DATE  \
0  13995332247820  2021-07-26 12:51:00.000  2021-07-27 17:19:00.000   
1  12633977630399  2021-07-26 11:33:00.000  2021-07-27 17:21:00.000   
2  13164894301526  2021-07-27 21:14:00.000  2021-07-27 21:38:00.000   
3  14601142299512  2022-01-16 09:22:00.000  2022-01-16 09:25:00.000   
4  17207276046856  2023-09-12 14:49:00.000  2023-09-13 06:26:00.000   

       DATE REPORT ENTERED                  VERIFYING PHYSICIAN  \
0  2021-07-27 17:19:00.000            MUJLLI,MOHAMMAD IQAB MOHD   
1  2021-07-27 17:21:00.000  AL KHAWALDEH,KHALED MOHAMMAD GHAL H   
2  2021-07-27 21:38:00.000        NOFAL,ABDALLAH JAMIL ABDALLAH   
3  2022-01-16 09:25:00.000        NOFAL,ABDALLAH JAMIL ABDALLAH   
4  2023-09-13 06:26:00.000        NOFAL,ABDALLAH JAMIL ABDALLAH   

                      TRANSCRIPTIONIST       IMAGING LOCATION  \
0            MUJLLI,MOHAMMAD IQAB MOHD   KHCC 1_MAIN BUILDING   
1  AL KHAW

In [7]:
import pandas as pd
# Read the CSV file
df = pd.read_csv('radiology_results_10.csv')

# Display the column names
print("Column names in the dataset:")
print(df.columns.tolist())



Column names in the dataset:
['MRN', 'EXAM_DATE', 'PROCEDURE', 'REPORT', 'modality', 'body_region', 'comparison_date', 'target_lesions', 'non_target_lesions', 'new_lesions', 'overall_assessment', 'ICDO3_site', 'ICDO3_site_term', 'ICDO3_site_similarity', 'tumor_response', 'error']


In [3]:
import pandas as pd
df= pd.read_csv('data/Results.csv')

# Pull first 10 rows of the data
first_10_rows = df.head(10)

# Print first 10 REPORT entries
print("\nFirst 10 Reports:")
for i, report in enumerate(df['REPORT'].head(10), 1):
    print(f"\nReport #{i}:")
    print(report)
    print("-" * 80)  # Print separator line between reports



First 10 Reports:

Report #1:

Diagnosis: Lymphoma.



Reason: Follow-up.



Comparison: Comparison made to the CT part of PET CT scan dated 2 February 2021.





Findings:



Neck CT with IV contrast:



There are unchanged enlarged hypoattenuating left cervical lymph nodes the large

st seen at levels III measuring 1 cm in short axis.



There are unchanged mildly enlarged right supraclavicular lymph nodes, none exce

eding 1 cm in short axis.



There is no new cervical lymph node enlargement.



Chest CT with IV contrast:



There are unchanged multiple hypoattenuating conglomerate mediastinal lymph node

s are encasing the aortic arch and its major branches with no significant narrow

ing, for example the subcarinal lymph node measures 2 cm in short axis with no s

ignificant change.



There is no new intrathoracic lymph node enlargement.



There is no significant axillary lymph node enlargement.



The mild pericardial thickening/effusion appears less significant.



There is 

In [3]:
from pydantic import BaseModel, Field
from pydantic_ai import Agent, RunContext
import logging
import asyncio
import nest_asyncio
from dataclasses import dataclass

# Enable nested event loops
nest_asyncio.apply()

# Configure logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

class SymptomAssessment(BaseModel):
    """Simple symptom assessment model"""
    main_symptom: str = Field(description="The primary symptom reported")
    severity: str = Field(description="Severity level: mild, moderate, or severe")
    duration: str = Field(description="How long the symptom has been present")
    recommendations: str = Field(description="Basic recommendations")

@dataclass
class SimpleDeps:
    """Simple dependencies class"""
    pass

# Initialize agent
assessment_agent = Agent(
    "gemini-2.0-flash-exp",
    retries=3,
    deps_type=SimpleDeps,
    result_type=SymptomAssessment,
    system_prompt="""
    You are a medical symptom analyzer.
    Extract key information about symptoms and provide basic recommendations.
    Focus on:
    1. Main symptom
    2. Severity
    3. Duration
    4. Basic recommendations for care
    """
)

@assessment_agent.tool
async def check_severity(ctx: RunContext[SimpleDeps], symptom: str, description: str) -> str:
    """Analyze symptom severity based on description"""
    if "severe" in description.lower() or "intense" in description.lower():
        return "severe"
    elif "mild" in description.lower() or "slight" in description.lower():
        return "mild"
    else:
        return "moderate"

async def process_symptoms():
    test_case = """
    Patient reports a headache that started 2 days ago.
    The pain is intense and throbbing, particularly in the morning.
    They have tried over-the-counter pain medication with little relief.
    """
    
    try:
        print("Starting symptom assessment...")
        deps = SimpleDeps()
        result = await assessment_agent.run(test_case, deps=deps)
        
        print("\nAssessment Results:")
        print(f"Main Symptom: {result.data.main_symptom}")
        print(f"Severity: {result.data.severity}")
        print(f"Duration: {result.data.duration}")
        print(f"Recommendations: {result.data.recommendations}")
        
        return result.data
        
    except Exception as e:
        print(f"Error during assessment: {str(e)}")
        raise

def run_test():
    try:
        loop = asyncio.get_event_loop()
        return loop.run_until_complete(process_symptoms())
    except Exception as e:
        print(f"Error running test: {str(e)}")
        raise

if __name__ == "__main__":
    run_test()
else:
    # For notebook environment
    result = run_test()

DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete
DEBUG:httpcore.connection:connect_tcp.started host='generativelanguage.googleapis.com' port=443 local_address=None timeout=5 socket_options=None


Starting symptom assessment...


DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.anyio.AnyIOStream object at 0x000001F10A03AAD0>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x000001F10663CF40> server_hostname='generativelanguage.googleapis.com' timeout=5
DEBUG:httpcore.connection:start_tls.complete return_value=<httpcore._backends.anyio.AnyIOStream object at 0x000001F10A03B8B0>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Content-Type', b'application/json; charset=UTF-8'), (b'Vary', b'Origin'), (b'Vary', b'X-Origin'), (b'Vary', b'Referer'), (b'Content-Encoding', b


Assessment Results:
Main Symptom: headache
Severity: severe
Duration: 2 days
Recommendations: Given the severity and lack of response to over-the-counter medication, it's recommended to consult a healthcare provider for further evaluation and management.


In [17]:
from pydantic_ai.models import AnthropicModel

ImportError: cannot import name 'AnthropicModel' from 'pydantic_ai.models' (c:\Users\isult\OneDrive\Documents\radiology_extraction\venv\lib\site-packages\pydantic_ai\models\__init__.py)